In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def truss_solver(E,a,cc):
    x = []
    y = []
    matriz_f = []
    no = []
    elemento = []
    no_i = []
    no_f = []
    x_temp = []
    y_temp = []
    zerar = []

    #cc = nós que possuem condição de contorno (devem ser inseridas em ordem crescente)
    cond_contorno = cc 

    # Zerando os nós que possuem alguma condição de contorno
    for i in cond_contorno: 
        zerar.append(2 * i - 2)
        zerar.append(2 * i - 1)
    zerar.reverse()
    
    # Leitura e armazenamento do arquivo que possui os nós e suas posições  
    arq1 = open('matriz_malha_exemplo.txt', 'r')  
    arq1.seek(0, 0)  
    for linha1 in arq1:
        linha1 = linha1.strip()
        NO, X, Y = linha1.split(',')
        x.append(float(X))
        y.append(float(Y))
        no.append(int(NO))
    arq1.close() 

    # Leitura e armazenamento do arquivo que possui os elementos e seus respectivos nós
    arq2 = open('matriz_conectividade_exemplo.txt', 'r')  
    for linha2 in arq2:
        linha2 = linha2.strip()
        ELEMENTO, NO_I, NO_F = linha2.split(',')
        no_i.append(int(NO_I))
        no_f.append(int(NO_F))
        elemento.append(int(ELEMENTO))
    arq2.close()

     # Leitura e armazenamento do arquivo que possui as forças aplicadas e sua magnitude
    arq3 = open('forcas_exemplo.txt', 'r') 
    arq3.seek(0, 0) 
    for linha3 in arq3:
        linha3 = linha3.strip()
        F = linha3
        matriz_f.append(float(F))
    arq3.close()  

    matriz_conectividade = np.transpose(np.asarray([elemento, no_i, no_f]))
    matriz_malha = np.transpose(np.asarray[no, x, y])

    # Plotando a treliça antes da deformação
    k = 0
    for i in matriz_conectividade:
        x_temp = (matriz_malha[indice1][1], matriz_malha[indice2][1])
        y_temp = (matriz_malha[indice1][2], matriz_malha[indice2][2])

        plt.subplot(1, 2, 1)
        plt.plot(x_temp, y_temp, 'r')
        plt.grid(b=bool, which='both', axis='both', color='k', linestyle=':', linewidth=0.5)
        plt.title(' Treliça 01 - Original ')
        plt.xlabel('X [mm]')
        plt.ylabel('Y [mm]')
        k = k + 1
    
    dim = 2 * len(matriz_malha)  # diz a dimensão da matriz kglobal
    kglobal = np.zeros([dim, dim], float)  # cria uma matriz kglobal com zeros

    z=1
    # Função que calcula as matrizes de rigidez locais
    def ElemTrel(E, a, x1, y1, x2, y2):  
        l = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        CS, S = (x2 - x1) / l, (y2 - y1) / l
        MatrizRig = (E * a / l) * np.array(
            [[CS ** 2, CS * S, -CS ** 2, -CS * S], [CS * S, S ** 2, -CS * S, -S ** 2], [-CS ** 2, -CS * S, CS ** 2, CS * S],
             [-CS * S, -S ** 2, CS * S, S ** 2]])
        return MatrizRig

 
    k = 0
    g = []
    h = []
    l1=[]

    #Calcula as matrizes de rigidez globais e as insere na posição correta na matriz de rigidez global
    for j in matriz_conectividade:
        m = np.zeros(([dim, 4]))  
        ktemp = []
        indice1 = matriz_conectividade[k][1] - 1
        indice2 = matriz_conectividade[k][2] - 1
        ktemp.append((ElemTrel(E, a[k], x[indice1], y[indice1], x[indice2], y[indice2])))  

        L1=((x[indice2] - x[indice1]) ** 2 + (y[indice2] - y[indice1]) ** 2) ** 0.5
        l1.append(L1)
        n1 = 2 * indice1  
        n2 = (2 * indice1) + 1  
        n3 = 2 * indice2  
        n4 = (2 * indice2) + 1 
        
        m[n1][0] = 1  
        m[n2][1] = 1
        m[n3][2] = 1
        m[n4][3] = 1
        d = np.matmul(m, ktemp)
        g = np.matmul(d, np.transpose(m))
        h.append(g)
        k += 1
    k = 0
    
    for i in h:
        kglobal = kglobal + h[k]
        k+=1

    k_temp = kglobal
    matriz_f_temp = np.array(matriz_f)

    for i in zerar:
        k_temp = np.delete(k_temp, (i), axis=1)  # esse laço zera as colunas e linhas das condições de contorno  
        k_temp = np.delete(k_temp, i, axis=2)  # tanto da matriz kglobal quanto do vetor de forças, assim
        matriz_f_temp = np.delete(matriz_f_temp, i, axis=0)  # temos {k}x{u}={f} com o mesmo numero de eq e incognitas.
        
    matriz_f_temp = np.transpose(np.asarray(matriz_f_temp))
    k_temp_inv = np.array(np.linalg.inv(k_temp))

    # resolução do sistema linear
    u = np.dot( matriz_f_temp,k_temp_inv)  

    # desempacotando a matriz_f_temp para uma lista
    q = []
    k = 0
    for i in matriz_f_temp:
        q.append(i)
        k += 1
    matriz_f_temp = q

    # desempacotando os deslocamentos para uma lista
    b = []
    k = 0
    for i in u:
        b.append(i)
        k += 1
    u = b
    u_final = []

    for k in range(dim):
        if k in zerar:
            u_final.append(0)
        else:
            u_final.append(u[0][0])
            u = np.delete(u, (0), axis=1)

    #Calculando as reações finais
    f_final = np.dot(kglobal, u_final)
 
    l2=[]
    k = 0
    for i in matriz_conectividade: 
        indice1 = matriz_conectividade[k][1] - 1
        indice2 = matriz_conectividade[k][2] - 1

        x_temp = (matriz_malha[indice1][1] + (u_final[indice1 * 2])*z,
                  matriz_malha[indice2][1] + (u_final[indice2 * 2] )*z)
        y_temp = (matriz_malha[indice1][2] + (u_final[indice1 * 2 + 1] )*z,
                  matriz_malha[indice2][2] + (u_final[indice2 * 2 + 1] )*z)
        #print(x_temp)
        L2 = ((x_temp[1] - x_temp[0]) ** 2 + (y_temp[1] - y_temp[0]) ** 2) ** 0.5
        l2.append(L2)
        plt.plot()
        plt.plot(x_temp, y_temp, 'b')
        plt.grid(b=bool, which='both', axis='both', color='k', linestyle=':', linewidth=0.5)
        plt.title(' Treliça 01 - Deformada ')
        plt.xlabel('X [m]')
        plt.ylabel('Y [m]')
        k = k + 1


    # Seleciona o maior deslocamento em módulo
    u_final_abs = [abs(ele) for ele in u_final]
    des_max = max(u_final_abs)

    # Seleciona a maior tensão em módulo
    l1 = np.array(l1)
    l2=np.array(l2)
    deformacao=(l1-l2)/l1
    tensao=deformacao*E
    tensao_abs = [abs(ele) for ele in tensao]
    tensao_max=max(tensao_abs)

    plt.show()
    return  des_max, tensao_max, 

